In [1]:
aws_access_key_id = 'i_have_changed'
aws_secret_access_key = 'and_this_also'
# https://t-redactyl.io/blog/2020/08/reading-s3-data-into-a-spark-dataframe-using-sagemaker.html
# good article

In [2]:
import json
data = { 'aws_access_key_id' : 'i_have_changed',
         'aws_secret_access_key' : 'and_this_also'
       }

with open('config.json','w') as fp:
    json.dump(data,fp,indent=4)
    

In [3]:
with open('config.json') as fp:
          data = json.load(fp)

In [5]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
import sagemaker_pyspark
import botocore.session

session = botocore.session.get_session()
credentials = session.get_credentials()

conf = (SparkConf()
        .set("spark.driver.extraClassPath", ":".join(sagemaker_pyspark.classpath_jars())))

spark1 = (
    SparkSession
    .builder
    .config(conf=conf) \
    .config('fs.s3a.access.key', credentials.access_key)
    .config('fs.s3a.secret.key', credentials.secret_key)
    .appName("ML_shubham")
    .getOrCreate()
)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/ml/linalg/__init__.py:81: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.22.4)
  import scipy.sparse
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/10/11 04:23:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
spark = SparkSession.builder \
        .appName("ML_shubham") \
        .config(conf=conf) \
        .config('fs.s3a.access.key', data['aws_access_key_id'])\
        .config('fs.s3a.secret.key', data['aws_secret_access_key'])\
        .getOrCreate()

24/10/11 04:23:44 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [7]:
bucket = "shubhnaa"
data_key = "prod.csv"
data_location = f"s3a://{bucket}/{data_key}"

df = spark.read.csv(data_location, header = 'True', inferSchema = True)

df.limit(5).toPandas()

24/10/11 04:23:46 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


,prdid,name,price,category
0,1.0,Widget A,25.0,Electronics
1,2.0,Widget B,NaN,Electronics
2,NaN,None,25.0,Electronics
3,NaN,None,NaN,None
4,3.0,Widget C,30.0,Clothing


In [10]:
df

DataFrame[prdid: int, name: string, price: double, category: string]

In [9]:
df.show()

+-----+--------+-----+-----------+
|prdid|    name|price|   category|
+-----+--------+-----+-----------+
|    1|Widget A| 25.0|Electronics|
|    2|Widget B| null|Electronics|
| null|    null| 25.0|Electronics|
| null|    null| null|       null|
|    3|Widget C| 30.0|   Clothing|
|    4|    null| 20.0|     Sports|
| null|Widget E| null|   Clothing|
|    6|Widget F| 35.0|       null|
| null|Widget G| null|     Sports|
|    8|    null| 30.0|Electronics|
|    9|        | 40.0|   Clothing|
|   10|    null| null|Electronics|
| null|    null| null|           |
+-----+--------+-----+-----------+



In [11]:
df.dtypes

[('prdid', 'int'),
 ('name', 'string'),
 ('price', 'double'),
 ('category', 'string')]

In [12]:
df.na.drop().show()

+-----+--------+-----+-----------+
|prdid|    name|price|   category|
+-----+--------+-----+-----------+
|    1|Widget A| 25.0|Electronics|
|    3|Widget C| 30.0|   Clothing|
|    9|        | 40.0|   Clothing|
+-----+--------+-----+-----------+



In [18]:
df.na.drop(how='any',thresh=3).show()

+-----+--------+-----+-----------+
|prdid|    name|price|   category|
+-----+--------+-----+-----------+
|    1|Widget A| 25.0|Electronics|
|    2|Widget B| null|Electronics|
|    3|Widget C| 30.0|   Clothing|
|    4|    null| 20.0|     Sports|
|    6|Widget F| 35.0|       null|
|    8|    null| 30.0|Electronics|
|    9|        | 40.0|   Clothing|
+-----+--------+-----+-----------+



In [25]:
df.na.drop(subset =['prdid']).show()

+-----+--------+-----+-----------+
|prdid|    name|price|   category|
+-----+--------+-----+-----------+
|    1|Widget A| 25.0|Electronics|
|    2|Widget B| null|Electronics|
|    3|Widget C| 30.0|   Clothing|
|    4|    null| 20.0|     Sports|
|    6|Widget F| 35.0|       null|
|    8|    null| 30.0|Electronics|
|    9|        | 40.0|   Clothing|
|   10|    null| null|Electronics|
+-----+--------+-----+-----------+



In [26]:
df.na.drop(subset =['prdid','name']).show()

+-----+--------+-----+-----------+
|prdid|    name|price|   category|
+-----+--------+-----+-----------+
|    1|Widget A| 25.0|Electronics|
|    2|Widget B| null|Electronics|
|    3|Widget C| 30.0|   Clothing|
|    6|Widget F| 35.0|       null|
|    9|        | 40.0|   Clothing|
+-----+--------+-----+-----------+



In [27]:
# drop column

In [28]:
df.drop('name').show()

+-----+-----+-----------+
|prdid|price|   category|
+-----+-----+-----------+
|    1| 25.0|Electronics|
|    2| null|Electronics|
| null| 25.0|Electronics|
| null| null|       null|
|    3| 30.0|   Clothing|
|    4| 20.0|     Sports|
| null| null|   Clothing|
|    6| 35.0|       null|
| null| null|     Sports|
|    8| 30.0|Electronics|
|    9| 40.0|   Clothing|
|   10| null|Electronics|
| null| null|           |
+-----+-----+-----------+



In [57]:
#using col
from pyspark.sql.functions import col,avg,min,max

In [31]:
df.drop(col('prdid')).show()

+--------+-----+-----------+
|    name|price|   category|
+--------+-----+-----------+
|Widget A| 25.0|Electronics|
|Widget B| null|Electronics|
|    null| 25.0|Electronics|
|    null| null|       null|
|Widget C| 30.0|   Clothing|
|    null| 20.0|     Sports|
|Widget E| null|   Clothing|
|Widget F| 35.0|       null|
|Widget G| null|     Sports|
|    null| 30.0|Electronics|
|        | 40.0|   Clothing|
|    null| null|Electronics|
|    null| null|           |
+--------+-----+-----------+



In [36]:
df.drop(col('name')).show()

+-----+-----+-----------+
|prdid|price|   category|
+-----+-----+-----------+
|    1| 25.0|Electronics|
|    2| null|Electronics|
| null| 25.0|Electronics|
| null| null|       null|
|    3| 30.0|   Clothing|
|    4| 20.0|     Sports|
| null| null|   Clothing|
|    6| 35.0|       null|
| null| null|     Sports|
|    8| 30.0|Electronics|
|    9| 40.0|   Clothing|
|   10| null|Electronics|
| null| null|           |
+-----+-----+-----------+



In [37]:
df.drop(df.category).show()

+-----+--------+-----+
|prdid|    name|price|
+-----+--------+-----+
|    1|Widget A| 25.0|
|    2|Widget B| null|
| null|    null| 25.0|
| null|    null| null|
|    3|Widget C| 30.0|
|    4|    null| 20.0|
| null|Widget E| null|
|    6|Widget F| 35.0|
| null|Widget G| null|
|    8|    null| 30.0|
|    9|        | 40.0|
|   10|    null| null|
| null|    null| null|
+-----+--------+-----+



In [38]:
col = ['prdid','name']
df.drop(col).show()

TypeError: col should be a string or a Column

In [39]:
col = ['prdid','name']
df.drop(*col).show()

+-----+-----------+
|price|   category|
+-----+-----------+
| 25.0|Electronics|
| null|Electronics|
| 25.0|Electronics|
| null|       null|
| 30.0|   Clothing|
| 20.0|     Sports|
| null|   Clothing|
| 35.0|       null|
| null|     Sports|
| 30.0|Electronics|
| 40.0|   Clothing|
| null|Electronics|
| null|           |
+-----+-----------+



In [40]:
# filling

In [41]:
df.show()

+-----+--------+-----+-----------+
|prdid|    name|price|   category|
+-----+--------+-----+-----------+
|    1|Widget A| 25.0|Electronics|
|    2|Widget B| null|Electronics|
| null|    null| 25.0|Electronics|
| null|    null| null|       null|
|    3|Widget C| 30.0|   Clothing|
|    4|    null| 20.0|     Sports|
| null|Widget E| null|   Clothing|
|    6|Widget F| 35.0|       null|
| null|Widget G| null|     Sports|
|    8|    null| 30.0|Electronics|
|    9|        | 40.0|   Clothing|
|   10|    null| null|Electronics|
| null|    null| null|           |
+-----+--------+-----+-----------+



In [45]:
df.fillna(value='shubham is great').show()   # string/categoric only

+-----+----------------+-----+----------------+
|prdid|            name|price|        category|
+-----+----------------+-----+----------------+
|    1|        Widget A| 25.0|     Electronics|
|    2|        Widget B| null|     Electronics|
| null|shubham is great| 25.0|     Electronics|
| null|shubham is great| null|shubham is great|
|    3|        Widget C| 30.0|        Clothing|
|    4|shubham is great| 20.0|          Sports|
| null|        Widget E| null|        Clothing|
|    6|        Widget F| 35.0|shubham is great|
| null|        Widget G| null|          Sports|
|    8|shubham is great| 30.0|     Electronics|
|    9|                | 40.0|        Clothing|
|   10|shubham is great| null|     Electronics|
| null|shubham is great| null|                |
+-----+----------------+-----+----------------+



In [48]:
df.fillna(value = 5.65436263000001).show() # numberic only

+-----+--------+----------------+-----------+
|prdid|    name|           price|   category|
+-----+--------+----------------+-----------+
|    1|Widget A|            25.0|Electronics|
|    2|Widget B|5.65436263000001|Electronics|
|    5|    null|            25.0|Electronics|
|    5|    null|5.65436263000001|       null|
|    3|Widget C|            30.0|   Clothing|
|    4|    null|            20.0|     Sports|
|    5|Widget E|5.65436263000001|   Clothing|
|    6|Widget F|            35.0|       null|
|    5|Widget G|5.65436263000001|     Sports|
|    8|    null|            30.0|Electronics|
|    9|        |            40.0|   Clothing|
|   10|    null|5.65436263000001|Electronics|
|    5|    null|5.65436263000001|           |
+-----+--------+----------------+-----------+



In [54]:
df.fillna(value = 435,subset=['category']).show()

+-----+--------+-----+-----------+
|prdid|    name|price|   category|
+-----+--------+-----+-----------+
|    1|Widget A| 25.0|Electronics|
|    2|Widget B| null|Electronics|
| null|    null| 25.0|Electronics|
| null|    null| null|       null|
|    3|Widget C| 30.0|   Clothing|
|    4|    null| 20.0|     Sports|
| null|Widget E| null|   Clothing|
|    6|Widget F| 35.0|       null|
| null|Widget G| null|     Sports|
|    8|    null| 30.0|Electronics|
|    9|        | 40.0|   Clothing|
|   10|    null| null|Electronics|
| null|    null| null|           |
+-----+--------+-----+-----------+



In [55]:
df.fillna(value = '435',subset=['category']).show()

+-----+--------+-----+-----------+
|prdid|    name|price|   category|
+-----+--------+-----+-----------+
|    1|Widget A| 25.0|Electronics|
|    2|Widget B| null|Electronics|
| null|    null| 25.0|Electronics|
| null|    null| null|        435|
|    3|Widget C| 30.0|   Clothing|
|    4|    null| 20.0|     Sports|
| null|Widget E| null|   Clothing|
|    6|Widget F| 35.0|        435|
| null|Widget G| null|     Sports|
|    8|    null| 30.0|Electronics|
|    9|        | 40.0|   Clothing|
|   10|    null| null|Electronics|
| null|    null| null|           |
+-----+--------+-----+-----------+



In [64]:
df.fillna(value = df.select(avg(col('price'))).collect()[0][0],subset='price').show()

+-----+--------+------------------+-----------+
|prdid|    name|             price|   category|
+-----+--------+------------------+-----------+
|    1|Widget A|              25.0|Electronics|
|    2|Widget B|29.285714285714285|Electronics|
| null|    null|              25.0|Electronics|
| null|    null|29.285714285714285|       null|
|    3|Widget C|              30.0|   Clothing|
|    4|    null|              20.0|     Sports|
| null|Widget E|29.285714285714285|   Clothing|
|    6|Widget F|              35.0|       null|
| null|Widget G|29.285714285714285|     Sports|
|    8|    null|              30.0|Electronics|
|    9|        |              40.0|   Clothing|
|   10|    null|29.285714285714285|Electronics|
| null|    null|29.285714285714285|           |
+-----+--------+------------------+-----------+



In [62]:
# col is a function
df.select(avg(col('price'))).collect()[0][0]

29.285714285714285

In [74]:
df.fillna(value = df.select(min(col('price'))).collect()[0][0]).show()

+-----+--------+-----+-----------+
|prdid|    name|price|   category|
+-----+--------+-----+-----------+
|    1|Widget A| 25.0|Electronics|
|    2|Widget B| 20.0|Electronics|
|   20|    null| 25.0|Electronics|
|   20|    null| 20.0|       null|
|    3|Widget C| 30.0|   Clothing|
|    4|    null| 20.0|     Sports|
|   20|Widget E| 20.0|   Clothing|
|    6|Widget F| 35.0|       null|
|   20|Widget G| 20.0|     Sports|
|    8|    null| 30.0|Electronics|
|    9|        | 40.0|   Clothing|
|   10|    null| 20.0|Electronics|
|   20|    null| 20.0|           |
+-----+--------+-----+-----------+



In [78]:
df.write.parquet('test.Parquet')

In [79]:
ls

config.json  test.parquet/  Untitled1.ipynb
lost+found/  test.Parquet/  Untitled.ipynb


In [80]:
cat test.parquet

cat: test.parquet: Is a directory


In [81]:
cd test.parquet

/home/ec2-user/SageMaker/test.parquet


In [82]:
ls

part-00000-1a43d5b1-1563-42e8-880f-f5ee4f95d93c-c000.snappy.parquet  _SUCCESS


In [84]:
bucket = "shubhnaa"
data_key = "test.parquet"
data_location = f"s3a://{bucket}/{data_key}"

In [85]:
df.write.parquet(data_location)

In [86]:
df1 = spark1.read.parquet(data_location)

In [87]:
df1.show()

+-----+--------+-----+-----------+
|prdid|    name|price|   category|
+-----+--------+-----+-----------+
|    1|Widget A| 25.0|Electronics|
|    2|Widget B| null|Electronics|
| null|    null| 25.0|Electronics|
| null|    null| null|       null|
|    3|Widget C| 30.0|   Clothing|
|    4|    null| 20.0|     Sports|
| null|Widget E| null|   Clothing|
|    6|Widget F| 35.0|       null|
| null|Widget G| null|     Sports|
|    8|    null| 30.0|Electronics|
|    9|        | 40.0|   Clothing|
|   10|    null| null|Electronics|
| null|    null| null|           |
+-----+--------+-----+-----------+

